In [1]:
import os,sys,inspect
from IPython.display import display, Markdown, Latex, Javascript, clear_output
import emoji
import time
import ipywidgets as widgets
import ipython_blocking
import numpy as np
import time
from tqdm import tqdm_notebook
import functools
import configparser
import webbrowser
import shutil

parentdir = os.path.abspath('../../../pymodules')

sys.path.insert(0,parentdir)
#from connect_device import connect
from connect_device_v2.vizualizate import vizulizated_finds_devices, advanced_search_device,DeviceBundle
from connect_device_v2.save_file import SaveFilesButton
from connect_device_v2.connect import connect_to_device
from connect_device_v2.find_device import FindDevice
from connect_device_v2.auxiliary_commands import check_error,disable_all_correctors,enable_ifbf_for_all_segment
import pandas as pd
from datetime import datetime
button = widgets.Button(
                description='Continue',
                disabled=False,
                button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
                tooltip='Click me',
                icon='play'        
                )

testname="Проверка устойчивости к механическому воздействию"
display(Markdown('### 💠 {}'.format(testname)))

### 💠 Проверка устойчивости к механическому воздействию

In [33]:
from connect_device_v2.find_device import FindDevice
box = FindDevice()
display(box)

<IPython.core.display.Javascript object>

FindDevice(children=(Dropdown(description='Выберите устройство:', layout=Layout(width='max-content'), options=…

In [34]:
button = box.select_device_continue_button

In [ ]:
%blockrun button

In [ ]:
button1 = box.select_config_continue_button

In [ ]:
%blockrun button1

In [ ]:
inst = box.selected_device

In [ ]:
inst.write("SYST:PRES")

In [ ]:
result_config =  box.config

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun 21 09:28:15 2021

@author: kuzma.m
"""

import pyvisa as visa

#функция декоратор
def command_wrapper(f):
    def wrapped(inst, command):
        # print('До функции')
        try:
            if key == "debug":
                print(command)
        except:
            pass
        response = f(inst, command)
        # print('После функции')
        error = inst.query('SYST:ERR?')
        if error != '0, No error':
            print(error)
        return response

    # print('декорируем', f)
    return wrapped

#запрос данных
@command_wrapper
def query(inst, command):
    result = inst.query(command)
    return result

@command_wrapper
def query_ascii_values(inst, command):
    result = inst.query_ascii_values(command)
    return result

#запрос данных
@command_wrapper
def write(inst, command):
    inst.write(command)

#Функция конвертации множителей kGz,MGh,GHz в герцы
def HzConvertor(n,k):
    if k == "kHz" or k == "кГц":
        return int(n*1e+3)
    elif k == "MHz" or k == "МГц":
        return int(n*1e+6)
    elif k == "GHz" or k == "ГГц":
        return int(n*1e+9)
    else: return -1

#Функция однократного росчерка
def singlescan(inst):
    write(inst, f'TRIG:SOUR BUS')
    write(inst, f'INIT:CONT OFF')
    write(inst, f'INIT')
    write(inst, f'TRIG:SING')
    query(inst, f'*OPC?')

In [ ]:
value_dict = {}

In [ ]:
def parse_config(config: dict):
    new_config = config.copy()
    
    if_split = config['DEVICE']['filter_if'].split(" ")
    if_value = f'{HzConvertor(int(if_split[0]),if_split[1])}'
    
    
    at_value = config['DEVICE']['athenuator_value']
    poin_value = config['DEVICE']['point_value']
    cap_value = config['DEVICE']['cap_value']
    
    
    new_config['DEVICE']['if_value']=if_value
    new_config['DEVICE']['athenuator_value']=at_value
    new_config['DEVICE']['point_value'] = poin_value
    new_config['DEVICE']['cap_value'] = cap_value
    return new_config
    

In [ ]:
new_config=parse_config(dict(result_config))

In [ ]:
def param(new_config):
    #Пресет
    write(inst, f'SYST:PRES')
    
    #Устанавливаем линейное сканирование
    write(inst, f'SENS:SWE:TYPE LIN')
    
    #Запрашиваем полный диапазон частот прибора и устанавливаем
    #freq_max = query(CMT, f'SERV:SWE:FREQ:MAX?')
    #freq_min = query(CMT, f'SERV:SWE:FREQ:MAX?')
    
    
    #Attenuator control – вкл. 
    write(inst, f'SERV:RFCTL:POW:STAT 1')
    write(inst, f'SERV:RFCTL:POW:ATT {new_config["DEVICE"]["athenuator_value"]}')

    #Установить фильтр ПЧ
    write(inst, f'SENS:BWID {new_config["DEVICE"]["if_value"]}')
    time.sleep(1) 

    #установить Код ЦАП.
    write(inst, f'SERV:RFCTL:POW:DAC {new_config["DEVICE"]["cap_value"]}')
    
    #установить кол-во точек.
    write(inst, f'SENS:SWE:POIN {new_config["DEVICE"]["point_value"]}')


In [ ]:
def setport():
    ports = query(inst,f'SERV:PORT:COUN?')
    write(inst, f'CALC1:PAR:COUN {ports}')
    for port in range(1,int(ports) + 1):
        write(inst, f'CALC1:PAR{port}:DEF S{port}{port}')

In [ ]:
def normalize():
    coun = query(inst,f'CALC1:PAR:COUN?')
    for trace in range(1,int(coun) + 1):
        write(inst,f'CALC:PAR{trace}:SEL')
        write(inst,f'CALC:MATH:MEM')
        write(inst,f'CALC:MATH:FUNC DIV')
    

In [ ]:
#Устанавливаем параметры и выставляем порты
param(new_config)
setport()

In [ ]:
#кнопка для выполнения нормализации
button_normalize = widgets.Button(description='Нормализировать', 
                        button_style='success' # 'success', 'info', 'warning', 'danger', ''
                        )
display(button_normalize)

In [ ]:
%blockrun button_normalize

In [ ]:
#нормализация
normalize()

In [ ]:
def marksearch(sost):
    value_dict.update({f'{sost}': {}})
    coun = query(inst,f'CALC1:PAR:COUN?')
    for trace in range(1,int(coun)+1):
        
        write(inst,f'CALC:PAR{trace}:SEL')
        write(inst, f'CALC1:TRAC{trace}:MARK1:STAT ON')
        write(inst, f'CALC1:TRAC{trace}:MARK2:STAT ON')
        write(inst, f'CALC:MARK1:FUNC:TYPE MAX')
        write(inst, f'CALC:MARK2:FUNC:TYPE MIN')
        
        write(inst, f'CALC:MARK1:FUNC:EXEC')
        write(inst, f'CALC:MARK2:FUNC:EXEC')
        
        max_value = abs(query_ascii_values(inst, f'CALC:MARK1:Y?')[0])
        min_value = abs(query_ascii_values(inst, f'CALC:MARK2:Y?')[0])
        if max_value > min_value:
            print(f'{trace} = {max_value}')
            value_dict[f'{sost}'].update({f'S{trace}{trace}': max_value})
        else:
            print(f'{trace} = {min_value}')
            value_dict[f'{sost}'].update({f'S{trace}{trace}': min_value})

In [ ]:
marksearch("before")
DSADSAD
marksearch("AFTER")